In [5]:
import urllib
import zipfile
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential

In [9]:
def preprocess(image, label):
    # NORMALIZE YOUR IMAGES HERE (HINT: Rescale by 1/.255)
    return tf.cast(image, tf.float32) / 255., label

In [10]:
def solution_model():
    # Downloads and extracts the dataset to the directory that
    # contains this file.

    IMG_SIZE = 128
    BATCH_SIZE = 8

    # The following code reads the training and validation data from their
    # respective directories, resizes them into the specified image size
    # and splits them into batches. You must fill in the image_size
    # argument for both training and validation data.
    # HINT: Image size is a tuple
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='train/',
        image_size=(128, 128),# YOUR CODE HERE
        batch_size=BATCH_SIZE)

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='validation/',
        image_size= (128,128),# YOUR CODE HERE
        batch_size=BATCH_SIZE)

    # Normalizes train and validation datasets using the
    # preprocess() function.
    # Also makes other calls, as evident from the code, to prepare them for
    # training.
    # Do not batch or resize the images in the dataset here since it's already
    # been done previously.
    train_ds = train_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(
        tf.data.experimental.AUTOTUNE)
    val_ds = val_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Code to define the model
    base = tf.keras.applications.efficientnet.EfficientNetB0(input_shape=(128,128,3),
                                                             weights='imagenet',
                                                             include_top=False)
    base.trainable = False
    model = tf.keras.Sequential([
        base,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])



    # Code to compile and train the model
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(0.0001),
                  metrics=['accuracy'])

    history2 = model.fit(
        train_ds,
        batch_size=BATCH_SIZE,
        validation_data=val_ds,
        epochs=10
    )

    return model

In [11]:
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

Found 10000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Epoch 1/10
1250/1250 [==============================] - 23s 15ms/step - loss: 0.6945 - accuracy: 0.4926 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/10
1250/1250 [==============================] - 19s 15ms/step - loss: 0.6942 - accuracy: 0.4984 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 3/10
1250/1250 [==============================] - 18s 14ms/step - loss: 0.6939 - accuracy: 0.5023 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/10
 617/1250 [=============>................] - ETA: 8s - loss: 0.6942 - accuracy: 0.4913